In [1]:
import pandas as pd

In [2]:
from sklearn.datasets import make_moons

In [3]:
## For 1000 Samples
n_samples = 1000

In [4]:
X,y = make_moons(n_samples=n_samples, noise=0.08, random_state=42)

In [5]:
circle_df = pd.DataFrame({"X1": X[:, 0], "X2": X[:, 1], "label": y})

In [6]:
import plotly.express as px

In [7]:
px.scatter(circle_df, x='X1', y='X2', color='label')

In [8]:
X.shape, y.shape

((1000, 2), (1000,))

## Turning Data Into Tensors

In [9]:
import torch

In [10]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [11]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) 

In [12]:
import torch
from torch import nn

In [13]:
device = "cpu"

## Model Construction

## Adding Non-Linearity to Neural Network

In [14]:
class BlobModel(nn.Module):
    """_summary_

    :param nn: _description_
    :type nn: _type_
    """

    def __init__(self, input_features, output_features, hidden_units=8):
        super(BlobModel, self).__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.Sigmoid(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.Sigmoid(), # <- does our dataset require non-linear layers? (try uncommenting and see if the results change)
            nn.Linear(in_features=hidden_units, out_features=output_features), # how many classes are there?
        )

        
    def forward(self, x):
        return self.linear_layer_stack(x)



## Loss Function & Optimizer

In [15]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [16]:
Model = BlobModel(input_features=2, output_features=1 , hidden_units=4).to(device)

In [17]:
Model.state_dict()

OrderedDict([('linear_layer_stack.0.weight',
              tensor([[ 0.2547, -0.4335],
                      [-0.0261,  0.3419],
                      [-0.5211, -0.5438],
                      [ 0.3998, -0.6378]])),
             ('linear_layer_stack.0.bias',
              tensor([-0.6927,  0.6033, -0.4500,  0.7013])),
             ('linear_layer_stack.2.weight',
              tensor([[ 0.0591, -0.2360,  0.1828, -0.4945],
                      [ 0.2859,  0.1485,  0.0992, -0.4586],
                      [-0.1501, -0.1576, -0.3801, -0.1832],
                      [-0.4294,  0.3177,  0.3299, -0.0818]])),
             ('linear_layer_stack.2.bias',
              tensor([ 0.3633,  0.0354,  0.4190, -0.2780])),
             ('linear_layer_stack.4.weight',
              tensor([[0.0419, 0.3220, 0.2429, 0.3659]])),
             ('linear_layer_stack.4.bias', tensor([-0.1090]))])

In [18]:
torch.manual_seed(42)
# Set the number of epochs
epochs = 10
device = "cpu"

# Put data to target device
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)


loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

# Create an optimizer
optimizer = torch.optim.SGD(params=Model.parameters(), 
                            lr=0.1)

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training
    Model.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = Model(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
    y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
  
    # 2. Calculate loss/accuracy
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    Model.eval()
    with torch.inference_mode():
        # 1. Forward pass
        test_logits = Model(X_test).squeeze() 
        test_pred = torch.round(torch.sigmoid(test_logits))
        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits,
                            y_test)
        test_acc = accuracy_fn(y_true=y_test,
                               y_pred=test_pred)

    



In [19]:
import numpy as np

In [20]:
labels = np.random.randint(2, size=100)  # binary label
predictions = np.random.rand(100)

In [22]:
predictions

array([0.10108827, 0.99457218, 0.29702012, 0.67079458, 0.33717153,
       0.80419663, 0.43831935, 0.03840446, 0.84523181, 0.0629228 ,
       0.25743598, 0.3556634 , 0.8576515 , 0.33326931, 0.85990745,
       0.1105077 , 0.20879337, 0.31939717, 0.86765213, 0.12930916,
       0.92088643, 0.00490131, 0.59196222, 0.48935302, 0.62967456,
       0.03893839, 0.57939187, 0.20280293, 0.06144454, 0.3176608 ,
       0.89375836, 0.50761978, 0.59381899, 0.50889389, 0.55720262,
       0.18589752, 0.8294477 , 0.28251367, 0.83165431, 0.63716342,
       0.73265237, 0.89341507, 0.84924759, 0.97793111, 0.30530917,
       0.35018382, 0.15923552, 0.53732898, 0.83123749, 0.91210153,
       0.19237638, 0.45656193, 0.86245646, 0.29087981, 0.78596594,
       0.32179592, 0.45838337, 0.76849918, 0.01646113, 0.46160603,
       0.01673081, 0.20998706, 0.55529926, 0.43211477, 0.99674197,
       0.59015368, 0.03509224, 0.75656503, 0.85250104, 0.54779441,
       0.36821148, 0.84143522, 0.13083543, 0.40672932, 0.63926

## Tensorboard to Visualize the Model Architecture

In [19]:
from torch.utils.tensorboard import SummaryWriter

In [20]:
id = 8

In [21]:
exp_id = f"MakeCircles_{id}"

In [22]:
writer = SummaryWriter(f"run/exp4/{exp_id}")

## Specifying A Model 

In [23]:
def TrainModel(epochs: int, X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test) -> None:
    """_summary_

    :param epochs: Number of Epoches
    :type epochs: int
    """

    torch.manual_seed(42)
    # Set the number of epochs
    device = "cpu"

    # Put data to target device
    X_train, y_train = X_train.to(device), y_train.to(device)
    X_test, y_test = X_test.to(device), y_test.to(device)

    Model = BlobModel(input_features=2, output_features=1 , hidden_units=4).to(device)

    loss_fn = nn.BCEWithLogitsLoss() # BCEWithLogitsLoss = sigmoid built-in

    # Create an optimizer
    optimizer = torch.optim.SGD(params=Model.parameters(), 
                                lr=0.05)

   # Build training and evaluation loop
    for epoch in range(epochs):
        ### Training
        Model.train()

        w_10 = Model.state_dict()['linear_layer_stack.0.weight'][0,0]
        w_20 = Model.state_dict()['linear_layer_stack.0.weight'][0,1]

        if epoch % 1000 == 0:

            wts_dict = {'W_10': w_10, 'W_20': w_20}
            writer.add_scalars('Weights Values across Epoch', wts_dict, epoch)

        # 1. Forward pass (model outputs raw logits)
        y_logits = Model(X_train).squeeze() # squeeze to remove extra `1` dimensions, this won't work unless model and data are on same device 
        y_pred = torch.round(torch.sigmoid(y_logits)) # turn logits -> pred probs -> pred labls
    
        # 2. Calculate loss/accuracy
        loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                       y_train) 
        acc = accuracy_fn(y_true=y_train, 
                          y_pred=y_pred)

        # writer.add_scalar(tag="Loss/Train", 
        #                   scalar_value=loss, 
        #                   global_step=epoch)

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backwards
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        ### Testing
        Model.eval()
        with torch.inference_mode():
            # 1. Forward pass
            test_logits = Model(X_test).squeeze() 
            test_pred = torch.round(torch.sigmoid(test_logits))
            # 2. Caculate loss/accuracy
            test_loss = loss_fn(test_logits,
                                y_test)
            
            # writer.add_scalar(tag="Loss/Test", 
            #                   scalar_value=test_loss, 
            #                   global_step=epoch)

            # We can Add Many Scalar Data into Single Plot using `add_scalars`
            if epoch % 1000 == 0:
                loss_dict = {'Train': loss, 'Test': test_loss}
                writer.add_scalars('Train Vs Test Loss', loss_dict, epoch)
            test_acc = accuracy_fn(y_true=y_test,
                                   y_pred=test_pred)

In [24]:
TrainModel(200000)

In [25]:
writer.add_graph(Model, input_to_model=X_train)

In [26]:
writer.flush()

In [27]:
writer.close()